In [1]:
import os

import numpy as np
import torch
from sklearn.model_selection import KFold
from torch.utils.data import SubsetRandomSampler, DataLoader

from src.data.data_preparer import DataPreparer
from src.data.data_reader import DataReader

while str(os.getcwd())[-3:] != 'src':  # Execute from src-directory root
    os.chdir('..')

In [2]:
businesses, reviews, tips = DataReader().read_data()
input_ml_train, input_ml_test, output_ml_train, output_ml_test = DataPreparer.get_train_test_validate(businesses, reviews, tips)

In [3]:
splits = KFold(n_splits=10, shuffle=True)


def train_epoch(model, device, dataloader, loss_fn, optimizer):
    train_loss = 0.0
    train_correct = 0
    model.train()
    for restaurant_reviews, ratings in dataloader:
        restaurant_reviews, ratings = restaurant_reviews.to(device), ratings.to(device)
        optimizer.zero_grad()
        output = model(restaurant_reviews)
        loss = loss_fn(output, ratings)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * restaurant_reviews.size(0)
        scores, predictions = torch.max(output.data, 1)
        train_correct += (ratings - 0.125 <= predictions <= ratings + 0.125).sum().item()

    return train_loss, train_correct


def valid_epoch(model, device, dataloader, loss_fn):
    valid_loss = 0.0
    val_correct = 0
    model.eval()
    for restaurant_reviews, ratings in dataloader:
        restaurant_reviews, ratings = restaurant_reviews.to(device), ratings.to(device)
        output = model(restaurant_reviews)
        loss = loss_fn(output, ratings)
        valid_loss += loss.item() * restaurant_reviews.size(0)
        scores, predictions = torch.max(output.data, 1)
        val_correct += (ratings - 0.125 <= predictions <= ratings + 0.125).sum().item()

    return valid_loss, val_correct

,business_average_stars_normalised,business_review_count_normalised,category_bakeries,category_coffee_&_tea,category_food,category_restaurants,category_sandwiches,category_burgers,category_ice_cream_&_frozen_yogurt,category_fast_food,...,attribute_divey,attribute_restaurantsattire,attribute_noiselevel,attribute_dessert,attribute_latenight,attribute_lunch,attribute_dinner,attribute_brunch,attribute_breakfast,average_checkins_per_week_normalised
1779609,0.625,0.326416,0,0,1,1,0,0,0,0,...,0.0,0.330078,0.5,0.0,0.0,1.0,1.0,0.5,0.0,0.060133
3113481,0.750,0.156494,0,0,0,1,0,0,0,0,...,0.0,0.330078,0.5,0.0,0.0,0.5,0.0,1.0,1.0,0.032306
1560302,0.625,0.014282,0,0,0,1,0,0,0,0,...,0.0,0.330078,0.5,0.0,0.0,0.0,1.0,0.0,0.0,0.001701
1104335,0.750,0.031204,0,1,1,1,0,0,0,0,...,0.0,0.330078,0.5,0.0,0.0,1.0,0.0,1.0,1.0,0.011114
2062337,0.625,0.015205,0,0,0,1,1,0,0,0,...,0.0,0.330078,0.5,0.0,0.0,1.0,0.0,0.0,0.0,0.003003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4545262,0.625,0.024063,0,0,0,1,0,0,0,0,...,0.0,0.330078,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.004146
2863797,0.875,0.053009,0,0,0,1,0,0,0,0,...,0.0,0.330078,0.5,0.5,0.0,0.5,1.0,1.0,0.5,0.007818
1037379,0.625,0.029221,0,0,0,1,0,0,0,0,...,0.0,0.330078,0.5,0.0,0.0,0.0,0.0,1.0,0.0,0.005026
551882,0.625,0.050903,0,0,0,1,0,0,0,0,...,0.0,0.330078,0.5,0.5,0.5,0.5,1.0,1.0,0.5,0.014974


In [ ]:
history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}

for i, train_idx_fold, val_idx_fold in enumerate(splits.get_n_splits(input_ml_train, input_ml_test)):

    train_data_fold = input_ml_train.iloc[train_idx_fold]
    validate_data_fold = input_ml_train.iloc[val_idx_fold]
    print('Fold {}'.format(i + 1))

    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

    model = ConvNet()
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.002)

    for epoch in range(num_epochs):
        train_loss, train_correct=train_epoch(model,device,train_loader,criterion,optimizer)
        test_loss, test_correct=valid_epoch(model,device,test_loader,criterion)

        train_loss = train_loss / len(train_loader.sampler)
        train_acc = train_correct / len(train_loader.sampler) * 100
        test_loss = test_loss / len(test_loader.sampler)
        test_acc = test_correct / len(test_loader.sampler) * 100

        print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Test Loss:{:.3f} AVG Training Acc {:.2f} % AVG Test Acc {:.2f} %".format(epoch + 1,
                                                                                                                                 num_epochs,
                                                                                                                                 train_loss,
                                                                                                                                 test_loss,
                                                                                                                                 train_acc,
                                                                                                                                 test_acc))
        history['train_loss'].append(train_loss)
        history['test_loss'].append(test_loss)
        history['train_acc'].append(train_acc)
        history['test_acc'].append(test_acc)